# 9주차 개인톡 과제
- 60171100 윤민혁

In [96]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import warnings
warnings.filterwarnings('ignore')
import numpy as np

## 적은 데이터로 모델 제작
- 영화 태극기 휘날리며

In [97]:
# 평점 높은순과 평점 낮은순
pres = ['https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=36666&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=highest&page=',
       'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=36666&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=lowest&page=']

review=[]
rate=[]
target=[]
# 각각 300개의 평점 크롤링
for pre in pres:
    for i in range(1,31):
        site = pre + str(i)
        res = requests.get(site)
        soup = BeautifulSoup(res.content,'html.parser')
    
        id_list = []
        id_pre = '_filtered_ment_'
    
        for i in range(10):
            id_list.append(id_pre+str(i))
    
        for id in id_list:
            review.append(soup.find('span',{'id':id}).get_text().strip())
        
        rate_list = []
        rate_list = (soup.select('div.star_score > em'))
    
        for r in rate_list:
            r = int(re.sub('<.+?>', '',str(r)))
            rate.append(r)
            if r>8:
                target.append(1)
            elif r<=4:
                target.append(0)
            else:
                target.append(-1)

In [98]:
df_1000 = pd.DataFrame({'review':review,'rate':rate,'target':target})
df_1000

,review,rate,target
0,이 영화는 진짜 잘만들었다 감동그자체 . 영화,10,1
1,,10,1
2,고등학생때 질질짜면서 보던게 어느세 30대라는 나이가 돼어 슬슬 40대가 되어갈 준...,10,1
3,"진짜 최고의 영화.. 전쟁에 대해서, 인간에 대해서 다시 생각해보게 됨",10,1
4,"이런 영화가 많이 나와야 하는데...6.25 참전용사님들, 자유민주주의 대한민국을 ...",10,1
...,...,...,...
595,이게머가재밌다는건지-_-,1,0
596,쿨럭!!...이영화 분수에 맞지 않게 넘 점수가 높다.,1,0
597,왜 빵점은 없는 것이냐~ 제작산지 배포산지 하는 짓거리가 맘에 들지 않아,1,0
598,재미없었어요. 진짜 봤구요. 제 갠적인 생각이니 태클마세요.,1,0


In [99]:
df_1000.rate.unique()

array([10,  1], dtype=int64)

In [100]:
# 데이터 셋 분리
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(review, target, test_size=0.2, random_state=0)

In [101]:
len(train_x), len(train_y)

(480, 480)

In [102]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

okt = Okt()
tfv = TfidfVectorizer(tokenizer=okt.morphs, ngram_range=(1,2), min_df=3, max_df = 0.9)
tfv.fit(train_x)

tfv_train_x = tfv.transform(train_x)
tfv_train_x

<480x603 sparse matrix of type '<class 'numpy.float64'>'
	with 4874 stored elements in Compressed Sparse Row format>

In [103]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

clf = LogisticRegression(random_state=0)
params = {'C':[1,3,5,7,9]}
grid_cv = GridSearchCV(clf, param_grid=params, cv = 4, scoring='accuracy', verbose=1)
grid_cv.fit(tfv_train_x, train_y)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


GridSearchCV(cv=4, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [1, 3, 5, 7, 9]}, scoring='accuracy', verbose=1)

In [104]:
tfv_test_x = tfv.transform(test_x)
grid_cv.best_estimator_.score(tfv_test_x,test_y)

0.7916666666666666

In [105]:
my_review1 = ['시간아깝네요','스토리 전개가 고구마 먹는 듯 답답','마지막까지 손에 땀을 쥐며 잘 봤습니다',
          '도무지 주제도 모르겠고 멀 전달하려는지 모르겠음','별로 추천하고 싶진 않네요',
          '몰입도가 높은 영화입니다. 시간가는 줄 몰랐네요.','곱씹을 수록 잘 만들어진 영화인 것 같아요. 다시 볼 겁니다.',
         '전개가 어이가 없네','평점 알바 왜이렇게 많냐','일상의 고단함을 잊게 해주었던 인생영화']
my_review2 = tfv.transform(my_review1)
for i in my_review2:
    if(grid_cv.best_estimator_.predict(i)):
         print('긍정 리뷰')
    else:
         print('부정 리뷰')


부정 리뷰
부정 리뷰
긍정 리뷰
부정 리뷰
부정 리뷰
부정 리뷰
긍정 리뷰
부정 리뷰
부정 리뷰
긍정 리뷰


In [106]:
my_review1 = ['시간아깝네요','스토리 전개가 고구마 먹는 듯 답답','마지막까지 손에 땀을 쥐며 잘 봤습니다',
          '도무지 주제도 모르겠고 멀 전달하려는지 모르겠음','별로 추천하고 싶진 않네요',
          '몰입도가 높은 영화입니다. 시간가는 줄 몰랐네요.','곱씹을 수록 잘 만들어진 영화인 것 같아요. 다시 볼 겁니다.',
         '전개가 어이가 없네','평점 알바 왜이렇게 많냐','일상의 고단함을 잊게 해주었던 인생영화']
my_review2 = tfv.transform(my_review1)
grid_cv.best_estimator_.predict(my_review2)

array([0, 0, 1, 0, 0, 0, 1, 0, 0, 1])

# 많은 데이터로 모델 제작

In [107]:
# 평점 높은순과 평점 낮은순
pres = ['https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=36666&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=highest&page=',
       'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=36666&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=lowest&page=']

review=[]
rate=[]
target=[]
# 각각 5000개의 평점 크롤링
for pre in pres:
    for i in range(1,501):
        site = pre + str(i)
        res = requests.get(site)
        soup = BeautifulSoup(res.content,'html.parser')
    
        id_list = []
        id_pre = '_filtered_ment_'
    
        for i in range(10):
            id_list.append(id_pre+str(i))
    
        for id in id_list:
            review.append(soup.find('span',{'id':id}).get_text().strip())
        
        rate_list = []
        rate_list = (soup.select('div.star_score > em'))
    
        for r in rate_list:
            r = int(re.sub('<.+?>', '',str(r)))
            rate.append(r)
            if r>8:
                target.append(1)
            elif r<=4:
                target.append(0)
            else:
                target.append(-1)

In [108]:
df_5000 = pd.DataFrame({'review':review,'rate':rate,'target':target})
df_5000

,review,rate,target
0,이 영화는 진짜 잘만들었다 감동그자체 . 영화,10,1
1,,10,1
2,고등학생때 질질짜면서 보던게 어느세 30대라는 나이가 돼어 슬슬 40대가 되어갈 준...,10,1
3,"진짜 최고의 영화.. 전쟁에 대해서, 인간에 대해서 다시 생각해보게 됨",10,1
4,"이런 영화가 많이 나와야 하는데...6.25 참전용사님들, 자유민주주의 대한민국을 ...",10,1
...,...,...,...
9995,흡인력 있는 서사. 전쟁장면도 좋지만 그보다 진한 형제간의 우정이 눈물을 자아내요,10,1
9996,짱이다!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!...,10,1
9997,개잼있음,10,1
9998,마이웨이보고 생각나서 왔는데 이런 작품이 한국에서 나왔다는게 새삼 대단하네요.,10,1


In [109]:
df_5000.rate.unique()

array([10,  1,  2,  3,  4,  5,  6,  7,  8,  9], dtype=int64)

In [110]:
train_x, test_x, train_y, test_y = train_test_split(review, target, test_size=0.2, random_state=0)

In [111]:
okt = Okt()
tfv = TfidfVectorizer(tokenizer=okt.morphs, ngram_range=(1,2), min_df=3, max_df = 0.9)
tfv.fit(train_x)

tfv_train_x = tfv.transform(train_x)
tfv_train_x

<8000x9220 sparse matrix of type '<class 'numpy.float64'>'
	with 136194 stored elements in Compressed Sparse Row format>

In [112]:
clf = LogisticRegression(random_state=0)
params = {'C':[1,3,5,7,9]}
grid_cv = GridSearchCV(clf, param_grid=params, cv = 4, scoring='accuracy', verbose=1)
grid_cv.fit(tfv_train_x, train_y)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


GridSearchCV(cv=4, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [1, 3, 5, 7, 9]}, scoring='accuracy', verbose=1)

In [113]:
tfv_test_x = tfv.transform(test_x)
grid_cv.best_estimator_.score(tfv_test_x,test_y)

0.8765

In [114]:
my_review1 = ['시간아깝네요','스토리 전개가 고구마 먹는 듯 답답','마지막까지 손에 땀을 쥐며 잘 봤습니다',
          '도무지 주제도 모르겠고 멀 전달하려는지 모르겠음','별로 추천하고 싶진 않네요',
          '몰입도가 높은 영화입니다. 시간가는 줄 몰랐네요.','곱씹을 수록 잘 만들어진 영화인 것 같아요. 다시 볼 겁니다.',
         '전개가 어이가 없네','평점 알바 왜이렇게 많냐','일상의 고단함을 잊게 해주었던 인생영화']
result = ['부정 리뷰','부정 리뷰','긍정 리뷰','부정 리뷰','부정 리뷰','긍정 리뷰','긍정 리뷰',
         '부정 리뷰','부정 리뷰','긍정 리뷰']
my_review2 = tfv.transform(my_review1)
grid_cv.best_estimator_.predict(my_review2)

array([ 0,  0,  1,  1, -1,  1,  1,  1,  0,  1])

In [115]:
for i in my_review2:
    if(grid_cv.best_estimator_.predict(i)):
         print('긍정 리뷰')
    else:
         print('부정 리뷰')

부정 리뷰
부정 리뷰
긍정 리뷰
긍정 리뷰
긍정 리뷰
긍정 리뷰
긍정 리뷰
긍정 리뷰
부정 리뷰
긍정 리뷰


- 평점 상위 300개와 하위 300개로 이루어진 첫번째 모델에서는 10개 중 9개에 대해 예측이 성공했다.
- 평점 상위 5000개와 하위 5000개로 이루어진 두번째 모델에서는 10개 중 7개에 대해 예측이 성공했다.
- 데이터가 많은 모델에서는 정확도는 올라갔지만 실제 리뷰를 적용했을 때 예측에 성공한 갯수는 데이터가 적은 모델이 조금 더 많은 것을 확인할 수 있었다.